# Session 3 — FastAPI & Pydantic

**Goals:**
- Understand why and how FastAPI uses Pydantic for validation
- Create request and response models with `BaseModel`
- Handle common validation scenarios and errors
- Learn `response_model`, `orm_mode`, nested models, and config

This notebook assumes minimal backend knowledge and explains concepts with real examples and runnable code.

## 1) Why Pydantic? (Simple explanation)

- Pydantic uses Python **type hints** to validate and coerce incoming data.
- It ensures the data shapes your endpoints expect — no more manual `if` checks.
- FastAPI uses Pydantic models to generate docs and validate requests automatically.

**Benefit for freshers:** fewer runtime bugs and clear API contracts.

## 2) Basic `BaseModel` example

Create models to describe the shape of JSON your API accepts and returns.

In [2]:
# app/schemas.py - basic models
from pydantic import BaseModel, Field
from typing import Optional

class ItemBase(BaseModel):
    name: str = Field(..., min_length=1, max_length=100, example='T-shirt') # ... means: this field is REQUIRED.
    description: Optional[str] = Field(None, example='100% cotton')

class ItemCreate(ItemBase):
    price: float = Field(..., gt=0, example=199.99)

class ItemRead(ItemBase):
    id: int
    price: float

    class Config:
        from_attributes = True  # allow from_orm conversions


### from_attributes = True  # allow from_orm conversions

`from_attributes = True` tells Pydantic that this model can be created from any Python object by **reading its attributes**, not just from plain dicts.

Concretely, with that config:

- You can do `ItemRead.model_validate(some_sqlalchemy_item)` and Pydantic will look at `some_sqlalchemy_item.id`, `.name`, `.description`, `.price` and use those attribute values to build the `ItemRead` instance.
- Without `from_attributes = True` (Pydantic v2) / `orm_mode = True` (v1), Pydantic expects a dict-like input (e.g. `{"id": 1, "name": "T-shirt", ...}`) and will not automatically pull values off an arbitrary object.

So that inner `Config` is essentially enabling the old “ORM mode” behavior for `ItemRead`, which is why the comment says “allow from_orm conversions”.

## 3) Using models in FastAPI endpoints

Use models as function parameters for request bodies and `response_model` for outputs. FastAPI validates incoming JSON automatically.

In [ ]:
# app/main.py - using models in endpoints
from fastapi import FastAPI, HTTPException
from typing import List
from app.schemas import ItemCreate, ItemRead

app = FastAPI(title='Pydantic Demo')

DB = {}
NEXT_ID = 1

@app.post('/items', response_model=ItemRead, status_code=201)
def create_item(item: ItemCreate):
    global NEXT_ID
    record = item.dict()
    record['id'] = NEXT_ID
    DB[NEXT_ID] = record
    NEXT_ID += 1
    return record

@app.get('/items', response_model=List[ItemRead])
def list_items():
    return list(DB.values())

@app.get('/items/{item_id}', response_model=ItemRead)
def get_item(item_id: int):
    item = DB.get(item_id)
    if not item:
        raise HTTPException(status_code=404, detail='Item not found')
    return item


## 4) What happens on validation error?

- FastAPI returns **422 Unprocessable Entity** with a helpful JSON body describing which field failed and why.
- This helps clients (frontend/mobile) fix requests quickly.

Try submitting invalid JSON (e.g., missing `price`) to see the error details in Swagger UI.

## 5) Nested models & lists

Models can contain other models or lists of models — useful for complex objects like orders.

In [ ]:
# Nested models example
from pydantic import BaseModel
from typing import List

class OrderItem(BaseModel):
    item_id: int
    quantity: int = 1

class OrderCreate(BaseModel):
    user_id: int
    items: List[OrderItem]

# Example usage in an endpoint
# @app.post('/orders', response_model=OrderRead)
# def create_order(order: OrderCreate): ...


## 6) Optional fields, defaults, and `Field()`

- Use `Optional[T]` or default values to make fields optional.
- `Field()` adds validation metadata (min/max, examples) and descriptions for docs.

In [ ]:
# Optional and default examples
from pydantic import BaseModel, Field
from typing import Optional

class UserBase(BaseModel):
    username: str = Field(..., min_length=3)
    bio: Optional[str] = None
    is_active: bool = Field(True, description='Active user flag')


## 7) `orm_mode` and ORMs (quick intro)

- `orm_mode = True` lets Pydantic `from_orm()` work with ORM objects (like SQLAlchemy models).
- This is useful when your DB returns objects instead of plain dicts.

You don't need full ORM knowledge right now — just know `orm_mode` helps convert DB objects to Pydantic models.

In [ ]:
# Example: pretend ORM object conversion (no SQLAlchemy required here)
class FakeORM:
    def __init__(self, id, name, price):
        self.id = id
        self.name = name
        self.price = price

item_orm = FakeORM(1, 'Demo', 99.9)
# Using the ItemRead model from earlier (assume imported)
from pydantic import BaseModel

class ItemRead(BaseModel):
    id: int
    name: str
    price: float

    class Config:
        orm_mode = True

model = ItemRead.from_orm(item_orm)
print(model.json())


## 8) `response_model` shaping and hiding fields

- Use `response_model` to control what is sent to the client.
- You can exclude sensitive fields (like internal notes) even if they exist on the DB record.

In [ ]:
# response_model include/exclude examples
from fastapi import FastAPI
from typing import List
from pydantic import BaseModel

class ItemRead(BaseModel):
    id: int
    name: str
    price: float

app = FastAPI()

# Suppose a DB record has extra internal fields
record = {'id':1, 'name':'Secret', 'price': 10.0, 'internal_note':'do not expose'}

@app.get('/public-item', response_model=ItemRead)
def public_item():
    # FastAPI will only include fields defined in ItemRead
    return record  # internal_note will be ignored in response


## 9) Field constraints and validators

- Use `Field(..., gt=0, le=100)` for numeric constraints.
- For advanced checks, use `@validator` or `@root_validator` on models.

In [ ]:
# Validators example
from pydantic import BaseModel, validator, root_validator

class Person(BaseModel):
    name: str
    age: int

    @validator('age')
    def age_must_be_positive(cls, v):
        if v < 0:
            raise ValueError('age must be >= 0')
        return v

class RangeModel(BaseModel):
    start: int
    end: int

    @root_validator
    def check_range(cls, values):
        s, e = values.get('start'), values.get('end')
        if s is None or e is None:
            return values
        if s > e:
            raise ValueError('start must be <= end')
        return values


## 10) Aliases, serialization, and `jsonable_encoder`

- Use `alias` to accept different JSON keys from clients.
- Use `jsonable_encoder` to convert Pydantic models (and other types) to JSON-serializable structures.

In [ ]:
# Alias and jsonable_encoder example
from pydantic import BaseModel, Field
from fastapi.encoders import jsonable_encoder

class Product(BaseModel):
    productName: str = Field(..., alias='product_name')

p = Product(product_name='Shoes')
print(p.dict())               # {'productName': 'Shoes'} (uses attribute name)
print(p.dict(by_alias=True))  # {'product_name': 'Shoes'} (uses alias)

# jsonable_encoder example
from datetime import datetime
from pydantic import BaseModel

class Event(BaseModel):
    name: str
    ts: datetime

e = Event(name='e', ts=datetime.utcnow())
print(jsonable_encoder(e))


## 11) Validation error structure (what the client sees)

FastAPI returns a structured JSON with details about each failed field. This is useful for frontend engineers to show precise errors.

In [ ]:
# Example of a validation error response body (illustrative)
example_error = {
    "detail": [
        {
            "loc": ["body", "price"],
            "msg": "field required",
            "type": "value_error.missing"
        }
    ]
}
print(example_error)


## 12) Global handling of validation errors (custom format)

You can customize how validation errors are returned by adding an exception handler for `RequestValidationError`.

In [ ]:
# custom_error_handler.py
from fastapi import FastAPI, Request
from fastapi.exceptions import RequestValidationError
from fastapi.responses import JSONResponse

app = FastAPI()

@app.exception_handler(RequestValidationError)
async def validation_exception_handler(request: Request, exc: RequestValidationError):
    return JSONResponse(status_code=422, content={
        "error": "Validation failed",
        "details": exc.errors()
    })


## 13) Settings management with `BaseSettings` (env/config)

Pydantic provides `BaseSettings` to load configuration from environment variables — useful for DB URLs, secrets (loaded securely in server env).

In [ ]:
# settings.py example
from pydantic import BaseSettings

class Settings(BaseSettings):
    app_name: str = 'MyApp'
    database_url: str

    class Config:
        env_file = '.env'

# Usage
# export DATABASE_URL='sqlite:///test.db'
# s = Settings()
# print(s.database_url)


## 14) Working with datetime, UUID and special types

Pydantic converts common types (datetime, date, UUID) from strings automatically.

In [ ]:
from datetime import datetime
from uuid import UUID
from pydantic import BaseModel

class ItemWithTs(BaseModel):
    id: int
    ts: datetime
    uid: UUID

# Example payload (client sends strings)
data = {'id':1, 'ts':'2025-01-01T12:00:00Z', 'uid':'123e4567-e89b-12d3-a456-426614174000'}
obj = ItemWithTs(**data)
print(obj)


## 15) Performance tips & common newbie mistakes

- Avoid heavy validators doing DB calls — keep validation lightweight.
- Use `response_model` to reduce accidental leakage of internal fields.
- Use `exclude_unset`, `exclude_defaults`, `exclude_none` when serializing models to control output.
- Prefer `Field(..., example=...)` to improve generated docs.

In [ ]:
# response model serialization options
from pydantic import BaseModel

class ItemRead(BaseModel):
    id: int
    name: str
    price: float

item = ItemRead(id=1, name='X', price=10.0)
print(item.dict())  # full dict
print(item.dict(exclude_none=True))  # exclude None fields


## 16) Exercises (Practical for freshers)

1. Create `ItemCreate`, `ItemRead` models and endpoints for POST and GET. Test validation by sending invalid payloads.
2. Build an `Order` model that nests `OrderItem` and validate quantities > 0.
3. Create a custom validator that enforces `start <= end` on a range model.
4. Add a custom global validation error handler that returns `{error: 'Validation failed', details: [...]}`.
5. Use `BaseSettings` to load a `DATABASE_URL` from `.env` and print it in `GET /config` endpoint.

---

When you're ready, I can also add a small SQLAlchemy example showing `from_orm()` conversion and a unit test using `pytest` + `httpx` for model validation.
